## MA 598 Programming Assignment 2:  Due Monday, February 25, 8pm.

### Model 1/2
Team member names:  Max Ruby, Carlos Salinas

In this assignment we use a GRU together with

* *Recurrent dropout*, a specific, built-in way to use dropout to fight overfitting in recurrent layers.

**Model 1: **  Modify the model described in Chollet, Section 6.3.6 (GRU plus reccurent dropout) as follows: 

- use a skip connection that adds the baseline prediction of the same temperature 24 hours ago to the output of the GRU, 
- use only 16 nodes in the GRU instead of 32, and 
- use a lookback of 432 (3 days).

**Model 2:** Extend Model 1 by encoding the date using 2 floating point numbers of the form (cos 2 π t/365, sin 2 π t/365), where t is the number of days from January 1. Likewise, encode the time of day using a similar format.  Include these values as input to the GRU + skip connection from Model 1. 


### To do:  
1. Specify the model number above and indicate the team member names.
- Specify your model structure below. 
- Restart kernel and run all.  
- Answer the questions in the next text box - in this assignment for Model 2 only.
- Print to pdf.
- Combine the pdfs for the 3 models into a single pdf.
- Submit on gradescope (indicate the start of each model and your team members on gradescope).  

### Questions (Model 2 only):  
Give brief (one or two sentence) answers to the questions below.

**Q1:** What other data preprocessing could you do that might improve accuracy? 

**Answer: **

**Q2: ** Why do you think that dropout didn't really improve the best test performance in the comparison in the original code from Section 6.3.6?  

**Answer: **


In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

## Recurrent neural networks for weather prediction

This notebook adapts the code samples found in Chapter 6, Section 3 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

---



## A temperature forecasting problem

This model uses weather 
timeseries dataset recorded at the Weather Station at the Max-Planck-Institute for Biogeochemistry in Jena, Germany: http://www.bgc-jena.mpg.de/wetter/.

In this dataset, fourteen different quantities (such air temperature, atmospheric pressure, humidity, wind direction, etc.) are recorded 
every ten minutes, over several years. The original data goes back to 2003, but we limit ourselves to data from 2009-2016. This dataset is 
perfect for learning to work with numerical timeseries. We will use it to build a model that takes as input some data from the recent past (a 
few days worth of data points) and predicts the air temperature 24 hours in the future.

First get access to the data:

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Look at the header and a few lines to understand the data better.  

In [3]:
import os

# data_dir = '/content/gdrive/My Drive/Datasets'
data_dir = os.getcwd()
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [24]:
print(lines[0])

01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.30


Convert all of these 420,551 lines of data into a Numpy array:

In [5]:
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
    

In [23]:
print(float_data[0])

[ 0.90014748 -1.93135845 -1.98211036 -1.86280029  1.07285236 -1.30742164
 -1.47375773 -0.79868641 -1.4762674  -1.47815522  2.12375056 -0.72950452
 -0.78067973 -0.27613603]


## Preparing the data


The exact formulation of our problem will be the following: given data going as far back as `lookback` timesteps (a timestep is 10 minutes) 
and sampled every `steps` timesteps, can we predict the temperature in `delay` timesteps?

We preprocess the data by subtracting the mean of each timeseries and dividing by the standard deviation. We plan on using the first 
200,000 timesteps as training data, so we compute the mean and standard deviation only on this fraction of the data:

In [7]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


Now here is the data generator that we will use. It yields a tuple `(samples, targets)` where `samples` is one batch of input data and 
`targets` is the corresponding array of target temperatures. It takes the following arguments:

* `data`: The original array of floating point data, which we just normalized in the code snippet above.
* `lookback`: How many timesteps back should our input data go.
* `delay`: How many timesteps in the future should our target be.
* `min_index` and `max_index`: Indices in the `data` array that delimit which timesteps to draw from. This is useful for keeping a segment 
of the data for validation and another one for testing.
* `shuffle`: Whether to shuffle our samples or draw them in chronological order.
* `batch_size`: The number of samples per batch.
* `step`: The period, in timesteps, at which we sample data. We will set it 6 in order to draw one data point every hour.

In [8]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


Now let's use our abstract generator function to instantiate three generators, one for training, one for validation and one for testing. 
Each will look at different temporal segments of the original data: the training generator looks at the first 200,000 timesteps, the 
validation generator looks at the following 100,000, and the test generator looks at the remainder.

In [22]:
lookback = 720
# lookback = 432
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (300000 - 200001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 300001 - lookback) // batch_size

## A recurrent model


Instead of the `LSTM` layer introduced in the previous section, we will use the `GRU` layer, developed by Cho et al. in 2014. `GRU` layers 
(which stands for "gated recurrent unit") work by leveraging the same principle as LSTM, but they are somewhat streamlined and thus cheaper 
to run, albeit they may not have quite as much representational power as LSTM. This trade-off between computational expensiveness and 
representational power is seen everywhere in machine learning.

The model below uses recurrent dropout to help prevent overfitting.  

In [37]:
from keras.models import Sequential
from keras import layers, Input, Model
from keras.optimizers import RMSprop

## Original model
# model = Sequential()
# model.add(layers.GRU(32,
#                      dropout=0.2,
#                      recurrent_dropout=0.2,
#                      input_shape=(None, float_data.shape[-1])))
# model.add(layers.Dense(1))

## Model 1
from keras.layers import GRU, Dense, Convolution2D, add


inputs = Input(shape=(None, 14))
print(keras.backend.shape(inputs))
x = GRU(16, dropout=0.2, recurrent_dropout=0.2)(inputs)
print(keras.backend.shape(x))
x = Dense(1)(x)
print(keras.backend.shape(x))
outputs = add([x, inputs])
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=5,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Tensor("Shape_2:0", shape=(3,), dtype=int32)
Tensor("Shape_3:0", shape=(2,), dtype=int32)
Tensor("Shape_4:0", shape=(2,), dtype=int32)
Epoch 1/5


ValueError: Error when checking target: expected add_12 to have 3 dimensions, but got array with shape (128, 1)

In [ ]:
from matplotlib import pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()